# 準備

In [ ]:
# !pip install gensim
# !pip install SudachiPy==0.5.4 ginza==4.0.6 ja-ginza==4.0.0 
# !pip install pytorch_lightning
# # !pip install emoji
# !pip install transformers[ja]
# !pip install sentencepiece

# !pip install fugashi
# !pip install ipadic

# !pip install deep_translator
!pip install googletrans

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 KB 8.7 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15735 sha256=d110c4f71df60f03c5108f34cf5f76d9aa34726907f1c8ddc42a98c65ae84581
  Stored in directory: /root/.cache/pip/wheels/27/f3/32/d4859d40071f07a5df0ab6fdc0076e78a8a786625dde2b4b2f
Successfully built googletrans
  Attempting uninstall: idna
    Found existing installation: idna 3.3
    Uninstalling idna-3.3

# import

In [29]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import os
import re
import matplotlib.pyplot as plt
from glob import glob
%matplotlib inline

#one_hotベクトル
from sklearn.feature_extraction.text import TfidfVectorizer

#ベクトル化
from gensim.models import word2vec

#データ分割用
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
import sklearn

import torch

from transformers import AutoTokenizer
from transformers import BertJapaneseTokenizer, AutoModelForMaskedLM, BertModel

# 翻訳
# -*- coding: utf-8
from googletrans import Translator

# データの読み込み

In [95]:
files = glob("*.xlsx")
files

['vocal_商品性情報.xlsx', 'tqnet情報.xlsx']

In [125]:
vocal_xlsx = files[0]
tqnet_xlsx = files[1]

vocal_df = pd.read_excel(vocal_xlsx)
tqnet_df = pd.read_excel(tqnet_xlsx)

vocal_df = vocal_df[['time', 'Unnamed: 1', 'Unnamed: 2']]
vocal_df.columns = ["time", "text", "car_name"]

tqnet_df.columns = ["car_name", "text"]

In [126]:

# tqnet_df
vocal_df

,time,text,car_name
0,2022-12-26,運転席のドアが閉めづらい\n運転席のドア内側の手をかける部分が高い位置の為ドアを閉めづらいそ...,シエンタ
1,2022-12-22,シフトノブ付近の小物入れが使いづらい\nシフトノブをパーキングに入れているとき小物入れとシフ...,ライズ
2,2022-12-22,助手席側のドアパネルとグローブボックスの位置が近い\n助手席側のドア下のポケットに厚手の物（...,ヴォクシー
3,2022-12-22,加速時のエンジン音が大きい\nアクセルを深く踏んだ時のエンジン音が大きい。特に力強く加速した...,ノア
4,2022-12-22,バックドアの開閉操作がやや使いづらい\nバックドアが開く時に上から手で押すとその位置で止まる...,ノア
5,2022-12-22,運転席 助手席降りる時開閉しづらい\nフロントドアトリムボードのドアノブの位置が高すぎて開閉...,シエンタ
6,2022-12-21,トランクを開けるスイッチの位置がわかりにくい。\nトランクを外から開けるスイッチの位置が車の...,クラウン
7,2022-12-17,メータ下にカードが入ってしまう。\nメータの所にカードを置いたら下に入って取れなくなった。\...,シエンタ
8,2022-12-03,シフトノブ部に物を落としてしまいそうな位隙間がある。\nシフトポジションをPレンジにした時に...,ヴォクシー
9,2022-11-30,バックドアが重くて開けにくい\n買い物帰りに荷物を積む時、バックドアが重くて大変とのご指摘を...,ノア


# データクリーニング
## vocal (X)

In [127]:
vocal_df["text"] = vocal_df["text"].str.replace("\n", ' ')
vocal_df["text"] = vocal_df["text"].str.replace("（\w+）", '')
vocal_df.head(12)

/tmp/ipykernel_55/3697679700.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  vocal_df["text"] = vocal_df["text"].str.replace("（\w+）", '')


,time,text,car_name
0,2022-12-26,運転席のドアが閉めづらい 運転席のドア内側の手をかける部分が高い位置の為ドアを閉めづらいそう...,シエンタ
1,2022-12-22,シフトノブ付近の小物入れが使いづらい シフトノブをパーキングに入れているとき小物入れとシフト...,ライズ
2,2022-12-22,助手席側のドアパネルとグローブボックスの位置が近い 助手席側のドア下のポケットに厚手の物入れ...,ヴォクシー
3,2022-12-22,加速時のエンジン音が大きい アクセルを深く踏んだ時のエンジン音が大きい。特に力強く加速した際...,ノア
4,2022-12-22,バックドアの開閉操作がやや使いづらい バックドアが開く時に上から手で押すとその位置で止まるが...,ノア
5,2022-12-22,運転席 助手席降りる時開閉しづらい フロントドアトリムボードのドアノブの位置が高すぎて開閉し...,シエンタ
6,2022-12-21,トランクを開けるスイッチの位置がわかりにくい。 トランクを外から開けるスイッチの位置が車の中...,クラウン
7,2022-12-17,メータ下にカードが入ってしまう。 メータの所にカードを置いたら下に入って取れなくなった。 物...,シエンタ
8,2022-12-03,シフトノブ部に物を落としてしまいそうな位隙間がある。 シフトポジションをPレンジにした時にシ...,ヴォクシー
9,2022-11-30,バックドアが重くて開けにくい 買い物帰りに荷物を積む時、バックドアが重くて大変とのご指摘を受...,ノア


## tqnet (Y)

In [128]:
tqnet_df["text"] = tqnet_df["text"].str.replace("\n", ' ')
tqnet_df["text"] = tqnet_df["text"].str.replace("�", '')
tqnet_df.head(6)

,car_name,text
0,ｼｴﾝﾀ,運転席に乗り込み内側からドアを閉めようとすると、手をかける場所が 上のほうにあるので非常に閉...
1,ﾙｰﾐｰ,走行中、加速した時エンジンの音が大きく感じて、段差などの時、カラカラと金属音がする。
2,ﾉｱ,シフトノブ部分の下側に隙間があり小銭などが入ってしまったら取れなくなってしまう。実際にほかの...
3,ｸﾗｳﾝ,トランクを開けるスイッチの場所がわかりずらく押しにくい
4,RAV4,運転席、助手席のドア取っ手が前寄りについているためドアを閉めるのが重い、強風が吹いたと き支...


# テキストベクトル化(辞書なし検討) - Tokenizer活用
## X_label creation (768次元)

In [129]:
# tokenizerとmodelのload
model_name = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model = model.cuda()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## 日本語を英語に変換

In [130]:
d_t = []
d_v = []
for t_data in tqnet_df["text"]:
    translated_t = GoogleTranslator().translate(t_data)
    d_t.append({
        "text": translated_t
    })
new_tqnet_df = pd.DataFrame(d_t)

for v_data in vocal_df["text"]:
    translated_v = GoogleTranslator().translate(v_data)
    d_v.append({
        "text": translated_v
    })
new_vocal_df = pd.DataFrame(d_v)

In [131]:
len(new_vocal_df["text"])

14

In [132]:
new_vocal_df.head(12)

,text
0,Driver's door is difficult to close It seems t...
1,It's hard to use the accessory compartment nea...
2,The door panel on the passenger side is close ...
3,The engine sound is loud when accelerating. Th...
4,Opening and closing the back door is a little ...
5,It is difficult to open and close when getting...
6,It is difficult to find the position of the sw...
7,A card is put under the meter. When I put the ...
8,There is a gap in the shift knob that seems to...
9,The back door is heavy and difficult to open W...


In [133]:
new_tqnet_df.head()

,text
0,When I got into the driver's seat and tried to...
1,"While driving, when I accelerate, I feel the e..."
2,There is a gap on the underside of the shift k...
3,It is difficult to find the switch to open the...
4,The door handles for the driver and front pass...


In [134]:
#各データの形式を整える
max_length = 256

x_dataset = []
# labels = []
for i in range(len(vocal_df)):
    # 記事から文章を抜き出し符号化を行う
    lines = new_vocal_df.iloc[i,0].splitlines()
    text = '\n'.join(lines)
    encoding = tokenizer(
        text,
        max_length = max_length,
        padding = 'max_length',
        truncation = True,
        return_tensors = 'pt'
        )
    encoding = {k: v.cuda() for k, v in encoding.items()}
    attention_mask = encoding['attention_mask']
    
    #文章ベクトルを計算
    with torch.no_grad():
        output = model(**encoding)
        last_hidden_state = output.last_hidden_state
        averaged_hidden_state =(last_hidden_state*attention_mask.unsqueeze(-1)).sum(1)/attention_mask.sum(1,keepdim=True) 
        
    #文章ベクトルとラベルを追加
    x_dataset.append(averaged_hidden_state[0].cpu().numpy())
    # label = vocal_df.iloc[i,1]
    # labels.append(label)

#ベクトルとラベルをnumpy.ndarrayにする
x_dataset = np.vstack(x_dataset)
# labels = np.array(labels)

## Y_label creation

In [135]:
max_length = 256

y_dataset = []
for i in range(len(tqnet_df)):
    # 記事から文章を抜き出し符号化を行う
    lines = new_tqnet_df.iloc[i,0].splitlines()
    text = '\n'.join(lines)
    encoding = tokenizer(
        text,
        max_length = max_length,
        padding = 'max_length',
        truncation = True,
        return_tensors = 'pt'
        )
    encoding = {k: v.cuda() for k, v in encoding.items()}
    attention_mask = encoding['attention_mask']
    
    #文章ベクトルを計算
    with torch.no_grad():
        output = model(**encoding)
        last_hidden_state = output.last_hidden_state
        averaged_hidden_state =(last_hidden_state*attention_mask.unsqueeze(-1)).sum(1)/attention_mask.sum(1,keepdim=True) 
        
    #文章ベクトルとラベルを追加
    y_dataset.append(averaged_hidden_state[0].cpu().numpy())

#ベクトルとラベルをnumpy.ndarrayにする
y_dataset = np.vstack(y_dataset)

# テキストのベクトル化(model)

In [136]:
print(x_dataset.shape)
print(y_dataset.shape)

(14, 768)
(5, 768)


In [137]:
import sklearn
data = sklearn.metrics.pairwise.cosine_similarity(x_dataset,y_dataset)

In [138]:
df = pd.DataFrame(data)

In [139]:
df.columns = ["Frドア重い", "加速音大きい", "シフト隙間", "SW場所不明確", "ドア重い"]

In [140]:
df["Vocal/TQ-NET"] = [
    "Frドア重い", 
    "シフト使用性", 
    "グローブボックス位置", 
    "加速音大きい", 
    "バックドア操作", 
    "Frドア開閉し難い", 
    "SW場所不明確", 
    "メータ隙間", 
    "シフト隙間", 
    "バックドア重い",
    "バックドア軽い",
    "Frドア操作性",
    "スーパー外れ値",
    "マルチメディア関係"
] 

In [141]:
first_column = df.pop("Vocal/TQ-NET")
df.insert(0,"Vocal / TQ-NET",first_column)

In [142]:
df.style.background_gradient()

,Vocal / TQ-NET,Frドア重い,加速音大きい,シフト隙間,SW場所不明確,ドア重い
0,Frドア重い,0.955876,0.848820,0.917601,0.861560,0.917336
1,シフト使用性,0.912677,0.847354,0.927047,0.858635,0.906873
2,グローブボックス位置,0.913321,0.832364,0.910594,0.853070,0.908063
3,加速音大きい,0.831902,0.861549,0.838009,0.781041,0.849909
4,バックドア操作,0.920419,0.845429,0.910918,0.883398,0.879734
5,Frドア開閉し難い,0.896679,0.804110,0.843189,0.860519,0.902255
6,SW場所不明確,0.906068,0.836625,0.907212,0.886353,0.892600
7,メータ隙間,0.909606,0.858724,0.935341,0.831867,0.877528
8,シフト隙間,0.875292,0.851176,0.916157,0.839909,0.885841
9,バックドア重い,0.910727,0.841409,0.916842,0.843307,0.899830
